# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("../WeatherPy/cities.csv")
weather_data

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Mataura,-46.1927,168.8643,43.56,92,35,2.08,NZ,1612367363
1,1,Albany,42.6001,-73.9662,30.99,72,100,3.00,US,1612367286
2,2,Greymouth,-42.4667,171.2000,56.61,88,13,6.11,NZ,1612367365
3,3,Ushuaia,-54.8000,-68.3000,53.60,50,40,26.46,AR,1612367367
4,4,Hofn,64.2539,-15.2082,33.80,64,75,11.50,IS,1612367368
...,...,...,...,...,...,...,...,...,...,...
572,572,Butajīra,8.1167,38.3667,64.09,27,12,8.43,ET,1612368016
573,573,Departamento de Maldonado,-34.6667,-54.9167,75.00,83,90,13.80,UY,1612368051
574,574,Deputatsky,69.3000,139.9000,-24.75,88,71,1.36,RU,1612368052
575,575,Copiapó,-27.3667,-70.3333,64.40,68,40,8.05,CL,1612368053


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

locations = weather_data[["Lat", "Lng"]]

humidity = weather_data["Humidity"]

In [4]:
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
new_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80)].dropna()

new_city_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
54,54,Thongwa,16.7619,96.5278,75.20,78,0,3.44,MM,1612367431
61,61,Bredasdorp,-34.5322,20.0403,73.40,53,0,9.22,ZA,1612367439
72,72,Port Alfred,-33.5906,26.8910,77.00,82,0,3.00,ZA,1612367451
98,98,Salalah,17.0151,54.0924,75.20,50,0,3.44,OM,1612367473
139,139,Hermanus,-34.4187,19.2345,75.99,50,0,5.01,ZA,1612367527
281,281,Abu Dhabi,24.4667,54.3667,71.60,68,0,6.91,AE,1612367604
313,313,Kiryat Gat,31.6100,34.7642,71.01,53,0,1.99,IL,1612367723
314,314,Vuyyūru,16.3667,80.8500,73.40,88,0,3.78,IN,1612367725
431,431,Plettenberg Bay,-34.0527,23.3716,70.93,74,0,5.68,ZA,1612367879
470,470,Muddebihāl,16.3333,76.1333,74.89,23,0,8.43,IN,1612367923


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = new_city_df.loc[:,["City","Country", "Lat", "Lng"]]

hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Lat,Lng,Hotel Name
54,Thongwa,MM,16.7619,96.5278,
61,Bredasdorp,ZA,-34.5322,20.0403,
72,Port Alfred,ZA,-33.5906,26.8910,
98,Salalah,OM,17.0151,54.0924,
139,Hermanus,ZA,-34.4187,19.2345,
281,Abu Dhabi,AE,24.4667,54.3667,
313,Kiryat Gat,IL,31.6100,34.7642,
314,Vuyyūru,IN,16.3667,80.8500,
431,Plettenberg Bay,ZA,-34.0527,23.3716,
470,Muddebihāl,IN,16.3333,76.1333,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [8]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
#     time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 54: Thongwa.
Missing field/result... skipping.
------------
Retrieving Results for Index 61: Bredasdorp.
Closest hotel in Bredasdorp is Bredasdorp Country Manor.
------------
Retrieving Results for Index 72: Port Alfred.
Closest hotel in Port Alfred is Royal St Andrews Hotel.
------------
Retrieving Results for Index 98: Salalah.
Closest hotel in Salalah is InterCityHotel Salalah.
------------
Retrieving Results for Index 139: Hermanus.
Closest hotel in Hermanus is Ocean Eleven.
------------
Retrieving Results for Index 281: Abu Dhabi.
Closest hotel in Abu Dhabi is Emirates Palace.
------------
Retrieving Results for Index 313: Kiryat Gat.
Closest hotel in Kiryat Gat is Desert Gat.
------------
Retrieving Results for Index 314: Vuyyūru.
Missing field/result... skipping.
------------
Retrieving Results for Index 431: Plettenberg Bay.
Closest hotel in Plettenberg Bay is The Plettenberg Hotel.
------------
Retrieving Results for Index 470: Muddebihāl.
Closest 

In [9]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
54,Thongwa,MM,16.7619,96.5278,
61,Bredasdorp,ZA,-34.5322,20.0403,Bredasdorp Country Manor
72,Port Alfred,ZA,-33.5906,26.8910,Royal St Andrews Hotel
98,Salalah,OM,17.0151,54.0924,InterCityHotel Salalah
139,Hermanus,ZA,-34.4187,19.2345,Ocean Eleven
281,Abu Dhabi,AE,24.4667,54.3667,Emirates Palace
313,Kiryat Gat,IL,31.6100,34.7642,Desert Gat
314,Vuyyūru,IN,16.3667,80.8500,
431,Plettenberg Bay,ZA,-34.0527,23.3716,The Plettenberg Hotel
470,Muddebihāl,IN,16.3333,76.1333,Gulbarga Hotel


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))